<img src="https://raw.githubusercontent.com/euroargodev/argopy/master/docs/_static/argopy_logo_long.png" alt="argopy logo" width="200"/>

# Training Camp - Sept 22<sup>th</sup> 2025

***

## Notebook Title : Compute your own per-profile diagnostic

**Author contact : [G. Maze](https://annuaire.ifremer.fr/cv/17182)**

**Description:**

This notebook shows how to complement an existing Argo data dataset with your own per-profile diagnostic.

The [Dataset.argo.reduce_profile()](https://argopy.readthedocs.io/en/v1.3.0/generated/xarray.Dataset.argo.reduce_profile.html#xarray.Dataset.argo.reduce_profile) method allows to execute a per profile diagnostic function very efficiently. 

Such a diagnostic function **takes vertical profiles as input and return a single value as output** (see examples below). Typical usage example would include computation of mixed layer depth or euphotic layer depth.

This is a notebook exploring this [section of the Argopy documentation](https://argopy.readthedocs.io/en/v1.3.0/user-guide/working-with-argo-data/data_computation.html#per-profile-custom-diagnostic).

*This notebook was developed with Argopy version: 1.3*

***

Let's start with some import:

In [ ]:
from argopy import DataFetcher
import numpy as np

And to prevent cell output to be too large, we won't display xarray object attributes:

In [ ]:
import xarray as xr
xr.set_options(display_expand_attrs = False)

### Execute a diagnostic without options

The most simple example is when the diagnostic method does not need arguments.

To illustrate this, let's apply a diagnostic that computes the mixed layer depth for a profile.

Such a diagnostic could be written as:

In [ ]:
def diag_mld(pres, sig0):
    """Return MLD with Boyer Montégut method with threshold of σ(10m) + 0.03 kg.m-3"""
    # Reference values
    threshold, threshold_depth = 0.03, 10.
    
    # Filter out NaN values
    idx = ~np.logical_or(np.isnan(pres), np.isnan(sig0))
    sig0_depth, sig0 = pres[idx], sig0[idx]

    # Check if there are valid data points near the reference depth
    if not np.any((sig0_depth >= 0) & (sig0_depth <= threshold_depth)):
        return np.nan

    # Get the reference density at the threshold depth
    index_threshold = np.argmin(np.abs(sig0_depth - threshold_depth))
    sig0_at_threshold = sig0[index_threshold]

    # Find the first depth where density exceeds the threshold
    exceeds_threshold = sig0[index_threshold:] > sig0_at_threshold + threshold
    if not np.any(exceeds_threshold):
        return np.nan

    mld_index = np.where(exceeds_threshold)[0][0] + index_threshold
    return sig0_depth[mld_index]

🛟 It is important to note that the per-profile diagnostic will receive 1-dimensional arrays of one profile data, not xarray object. 

Then to apply the diagnostic to a collection of Argo profile, let's first load data from a region just South of the Gulf Stream:

In [ ]:
%%time
f = DataFetcher().region([-69., -64., 32., 36., 0., 1000., '20200101', '20230101'])
ds = f.data
ds.argo

<br>

and convert this collection of points to a collection of profiles:

In [ ]:
dsp = ds.argo.point2profile()
dsp.argo

<br>

We note that the `diag_mld` method above, takes pressure and potential density as arguments, in this order.

Argopy and xarray will handle all the axis and dimensions manipulation, so that you can focus on writing a reducer function dealing with 1D arrays for each requested parameters.

So, to apply this diagnostic to the xarray Argo dataset, we first compute SIG0 and then call the reducer:

In [ ]:
%%time
dsp.argo.teos10(['SIG0']);

In [ ]:
dsp['MLD'] = dsp.argo.reduce_profile(diag_mld, params=['PRES', 'SIG0'])
dsp['MLD']

#### 🛟 Note

Note that the new variable has the appropriate dimension `N_PROF` and coordinates.


#### 🔍 Pro tip

You can add attributes to the new variable and you can plot it along the dataset potential density like this:

(easier methods shall be developped in the future)

In [ ]:
dsp['MLD'].attrs = {'unit': 'db',
                   'long_name': 'Mixed Layer Depth',
                   'method': 'Density threshold'}

In [ ]:
from argopy.plot import scatter_plot
import matplotlib.pyplot as plt

fig, ax, _, _ = scatter_plot(dsp, 'SIG0', cmap='Spectral_r', s=12, cbar=True);
ax.plot(dsp['TIME'], dsp['MLD'], 'k', label=dsp['MLD'].attrs['long_name'])
ax.legend(loc='lower left');

#### ✏️ EXERCICE

Write down a function computing the oxygen minimum depth and apply it to some BGC-Argo dataset.

💡 Code hint: 
```python
def min_oxygen_depth(pres, doxy):    
    idx = ~np.logical_or(np.isnan(pres), np.isnan(doxy))  # Filter out NaN values
    return ...

f = DataFetcher(ds='bgc', params='DOXY').float(6903222)
dsp = f.data.argo.point2profile()
dsp...
```

In [ ]:
# Your code

### Execute a diagnostic with options

In a more realistic scenario, it will be often the case that your diagnostic will require options to play around with some parameters.

**To add options to your diagnostic, just add them as named arguments to the method and provide them to the Argopy reducer.**

To illustrate this, let's take a diagnostic method that computes the depth of an isothermal surface, which should be given as an option.

It could go like this:

In [ ]:
def isoT_depth(pres, temp, iso = 12.):
    idx = ~np.logical_or(np.isnan(pres), np.isnan(temp))
    idx_top, idx_btm = temp[idx] >= iso, temp[idx] <= iso
    if np.any(idx_top):
        i_top, i_btm = np.max(np.argwhere(idx_top)), np.min(np.argwhere(idx_btm))
        t_top, p_top, t_btm, p_btm = temp[i_top], pres[i_top], temp[i_btm], pres[i_btm]
        return p_top + (p_btm-p_top)*(t_top-isotemp)/(t_top-t_btm)
    return np.nan

<br>

Let's download and transform some float data to illustrate this diagnostic:

In [ ]:
%%time
f = DataFetcher().float(6902915)  # A float in the North Atlantic subtropical gyre
dsp = f.data.argo.point2profile()
dsp.argo

<br>

Let's now apply the diagnostic:

In [ ]:
isotemp = 26.  # Isothermal surface target, in degree Celsius

da = dsp.argo.reduce_profile(isoT_depth, params=['PRES', 'TEMP'], iso=isotemp)
da.attrs = {'unit': 'db',
            'long_name': f"TEMP={isotemp}^oC depth"}

<br>

Add the new variable to the Argo dataset:

In [ ]:
dsp['isoT'] = da

#### 🔍 Pro tip

You can plot the new variable like this:

(easier methods shall be developped in the future)

In [ ]:
from argopy.plot import scatter_plot
import matplotlib.pyplot as plt

fig, ax, _, _ = scatter_plot(dsp, 'TEMP', cmap='Spectral_r', s=6, vmin=isotemp-2, vmax=isotemp+2, cbar=1);
ax.plot(dsp['TIME'], dsp['isoT'], 'k', label=dsp['isoT'].attrs['long_name'])
ax.set_ylim([np.nanmax(dsp['isoT'])+200, 0])
ax.legend(loc='lower left');

#### ✏️ EXERCICE

Getting back to the mixed layer depth diagnostic, the density reference level and threshold could be options easily accessible for a more appropriate diagnostic.

Modify the `diag_mld` method to take reference values as option and demonstrate its use.

In [ ]:
# Your code here

In [ ]:
# Your code here

### 
***
Useful argopy commands:
```python
argopy.reset_options()
argopy.show_options()
argopy.status()
argopy.clear_cache()
argopy.show_versions()
```
***
![logo](https://raw.githubusercontent.com/euroargodev/argopy-training/refs/heads/main/for_nb_producers/template_argopy_training_EAONE.png)